In [17]:
import requests as r
from bs4 import BeautifulSoup
from selenium import webdriver
from PIL import Image
from io import BytesIO

import os
import shutil
import re
import keyboard

In [18]:
DATA_PATH = './wallpaper-pictures'
farpost_url = 'https://www.farpost.ru'

In [19]:
driver = webdriver.Edge()

In [14]:
'''
Parsing method with waiting for captcha processing by user
- press q if can't pass the captcha and you want to skip this page
- press c if the captcha is missing or has been passed
'''

def wait_user_decision():
    while True:
        if keyboard.is_pressed('q'):
            return 0
        elif keyboard.is_pressed('c'):
            return 1


def selenium_parse(url):
    driver.get(url)
    res = driver.page_source

    soup = BeautifulSoup(res, 'html.parser')
    tags = soup.find_all()
    for tag in tags:
        if 'class' in tag.attrs and any(map(lambda cls: re.search(r'\b\w*cap(?:tcha)?\d*\b', cls), tag.attrs['class'])):
            des = wait_user_decision()
            return selenium_parse(url) if des else None

    return soup

In [41]:
'''
Getting all available wallpaper pictures
'''

url = 'https://www.farpost.ru/home/materials/ceiling/+/Обои/'
soup = selenium_parse(url)
options_table = soup.find('div', {'name': 'wallpaperPicture'})


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=126.0.2592.102)
Stacktrace:
	GetHandleVerifier [0x00007FF715798132+13538]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF715721DE9+595465]
	(No symbol) [0x00007FF71553E6CF]
	(No symbol) [0x00007FF71551B33A]
	(No symbol) [0x00007FF7155A82AD]
	(No symbol) [0x00007FF7155BAAFF]
	(No symbol) [0x00007FF7155A1C63]
	(No symbol) [0x00007FF71557766E]
	(No symbol) [0x00007FF71557683C]
	(No symbol) [0x00007FF715577221]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF7159596D4+1099860]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF71569D8FC+53532]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF715690E25+1605]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF715958665+1095653]
	Microsoft::Applications::Events::ILogConfiguration::opera

In [ ]:
'''
Getting pages urls with ads containing 
wallpapers with pictures of a certain type
'''

picture_options = options_table.find_all('a', {'class': 'option'})
picture_to_section_url = {}

for option in picture_options:
    picture_name = option.get('data-value')
    url = option.get('href')
    picture_to_section_url[picture_name] = url

list(picture_to_section_url.items())[:2]

[('bez_risunka',
  'https://www.farpost.ru/home/materials/ceiling/+/%D0%9E%D0%B1%D0%BE%D0%B8/?wallpaperPicture%5B%5D=bez_risunka'),
 ('abstrakciya',
  'https://www.farpost.ru/home/materials/ceiling/+/%D0%9E%D0%B1%D0%BE%D0%B8/?wallpaperPicture%5B%5D=abstrakciya')]

In [32]:
'''
Continue filling folders with images if 
previous session was interrupted
'''
def creation_time(file):
        return os.path.getctime(os.path.join(DATA_PATH, file))


if os.path.exists(DATA_PATH):
    existing_folders = os.listdir(DATA_PATH)
    start_idx = len(existing_folders)

    sorted_folders = sorted(existing_folders, key=creation_time)
    shutil.rmtree(DATA_PATH + '/' + sorted_folders[-1])
else:
    os.makedirs(DATA_PATH)
    start_idx = 0

In [39]:
'''
Getting urls of all ads by picture types
'''

picture_to_adv_urls = {}

for picture_name, url in list(picture_to_section_url.items())[start_idx:]:
    soup = selenium_parse(url)

    adv_urls = soup.find_all('a', {'class': 'bulletinLink'})
    adv_urls = list(map(lambda x: farpost_url + x.get('href'), adv_urls))
    picture_to_adv_urls[picture_name] = adv_urls

In [40]:
'''
Reading images from each ad and splitting into folders
'''

for picture_name, adv_urls in list(picture_to_adv_urls.items()):
    picture_dir = DATA_PATH + '/' + picture_name
    os.mkdir(picture_dir)

    for i, adv_url in enumerate(adv_urls):
        soup = selenium_parse(adv_url)
        if soup is None:
            continue
        
        images_container = soup.find('div', {'class': 'image-gallery__big-image-container'})
        if images_container is None:
            continue
        images = images_container.find_all('img', {'class': 'image-gallery__big-image'})
        images_urls = list(map(lambda x: x.get('src'), images))

        for j, image_url in enumerate(images_urls):
            res = r.get(image_url) # because there will be no captcha here
            image = Image.open(BytesIO(res.content))
            image.save(picture_dir + f'/{i}{j}.jpg')

In [ ]:
driver.quit()